<a href="https://colab.research.google.com/github/kailashkumar-S/kailashkumar-S/blob/main/Historical_VaR_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Historical Var Analysis

##### What is VaR?
Value at risk (VaR) is a measure of the risk of loss of investment/Capital. It estimates how much a set of investments might lose (with a given probability), given normal market conditions, in a set time period such as a day. VaR is typically used by firms and regulators in the financial industry to gauge the amount of assets needed to cover possible losses.

VaR has four main uses in finance:
- risk management
- financial control
- financial reporting
- computing regulatory capital.

A typical VaR curve:

![VaR curve](https://www.investopedia.com/thmb/-eYsp4xWm_s3y65C0T8cqvCeEsQ=/750x0/filters:no_upscale():max_bytes(150000):strip_icc()/Variance-CovarianceMethod5-5bde86ce7819405ca63f26aa275a4bd2.png)

In [1]:

!pip install yfinance

In [2]:

import datetime as dt
import pandas as pd
import numpy as np

from pandas_datareader import data as pdr
import yfinance as yf
import plotly.offline as offline
import plotly.graph_objects as go
from plotly.subplots import make_subplots

offline.init_notebook_mode(connected=True)
pd.options.plotting.backend = 'plotly'


In [3]:
end = dt.datetime.now()
start = dt.datetime(2023,1,1)

df = yf.download('SPY', start, end) # Getting the data of S&P 500 Index
df.head()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-01-03,384.369995,386.429993,377.829987,380.820007,373.956818,74850700
2023-01-04,383.179993,385.880005,380.000000,383.760010,376.843842,85934100
2023-01-05,381.720001,381.839996,378.760010,379.380005,372.542755,76970500
2023-01-06,382.609985,389.250000,379.410004,388.079987,381.085938,104189600
2023-01-09,390.369995,393.700012,387.670013,387.859985,380.869934,73978100


Log Return

- Log returns represent the continuosly compunded growth rate of an investment.
- If the returns are normally distributed, log returns have several statistical advantages, including a symmetric distribution, which simplifies mathematical and statistical analysis.

![Log Return Formula](https://latex.codecogs.com/svg.image?&space;R_{log}=ln(\frac{Price_{new}}{Price_{old}}))

In [4]:
#Calculating log returns

log_returns = np.log(df.Close/df.Close.shift(1)).dropna()
log_returns

Date
2023-01-04    0.007691
2023-01-05   -0.011479
2023-01-06    0.022673
2023-01-09   -0.000567
2023-01-10    0.006988
                ...   
2024-06-10    0.003085
2024-06-11    0.002405
2024-06-12    0.008179
2024-06-13    0.002011
2024-06-14    0.000608
Name: Close, Length: 364, dtype: float64

The annualized standard deviation of daily returns is calculated as follows:

- Annualized Standard Deviation = Standard Deviation of Daily Returns * Square Root (250)

- For weekly returns, Annualized Standard Deviation = Standard Deviation of Weekly Returns * Sqrt(52).

- For monthly returns, Annualized Standard Deviation = Standard Deviation of Monthly Returns * Sqrt(12).

- For quarterly returns, Annualized Standard Deviation = Standard Deviation of Quarterly Returns * Sqrt(4).

In [5]:
# Calculating daily standard deviation of returns

daily_std = log_returns.std()

annualized_std = daily_std*np.sqrt(252)


In [6]:
fig = make_subplots(rows=2,cols=2)

trace = go.Histogram(x=log_returns)

fig.update_layout(autosize = True,
                  width=500,
                  height=500,
                  title='log returns fequency curve',
                  xaxis = dict(title = 'Annualised standard deviation:' + str(np.round(annualized_std*100,2)))
                 )
fig.add_trace(trace, row=1, col=1)

fig.show()

- Calculate and visualize the rolling standard deviation of log returns over a specified window of trading days

In [7]:
TRADING_DAYS = 5

standard_deviation = log_returns.rolling(window=TRADING_DAYS).std()*np.sqrt(TRADING_DAYS)
standard_deviation.dropna().head()

standard_deviation.plot().show()

#### Sharpe Ratio

- The Sharpe ratio divides a portfolio's excess returns by a measure of its volatility to assess risk-adjusted performance
- Excess returns are those above an industry benchmark or the risk-free rate of return

![Sharpe Ratio](https://www.gstatic.com/education/formulas2/553212783/en/sharpe_ratio.svg)

$ S_{a}	=	Sharpe\;ratio $

$ E	=	expected\;value$

$ R_{a}	=	asset\;return$

$ R_{b}	=	risk\;free\;return$

$ \sigma_{a}	=	standard\;deviation\;of\;the\;asset\;excess\;return$


In [8]:
Rf = 0.01/TRADING_DAYS
sharpe_ratio = (log_returns.rolling(window=TRADING_DAYS).mean()-Rf)*TRADING_DAYS/standard_deviation
sharpe_ratio.plot().show()


#### Sortino Ratio

- The Sortino ratio is a variation of the Sharpe ratio that differentiates harmful volatility from total overall volatility by using the asset's standard deviation of negative portfolio returns—downside deviation—instead of the total standard deviation of portfolio returns.
- The Sortino ratio takes an asset or portfolio's return and subtracts the risk-free rate, and then divides that amount by the asset's downside deviation.

![](https://www.wallstreetmojo.com/wp-content/uploads/2022/08/Sortino-Ratio-Formula.jpg.webp)


In [9]:
sortino_std = log_returns[log_returns<0].rolling(window=TRADING_DAYS, center=True, min_periods=1).std()*np.sqrt(TRADING_DAYS)
sortino_ratio = (log_returns.rolling(window=TRADING_DAYS).mean()-Rf)*TRADING_DAYS/ sortino_std

#sortino_std.plot()

sortino_ratio.plot(kind='bar')

#### Modigliani Ratio (M2 Ratio)

Modigliani ratio shows the return on an investment adjusted for risk in comparison to a benchmark. It is shown as units of percentage return.

![](https://cdn.corporatefinanceinstitute.com/assets/risk-adjusted-return-ratios5.png)

Where:

$ R_{p} = Return\;on\;Adjusted\;Portfolio $

$ R_{m} = Return\;on\;Market\;Portfolio $

In [10]:
m2_ratio = pd.DataFrame()

benchmark_std = standard_deviation
standard_deviation = standard_deviation.dropna()
sharpe_ratio = sharpe_ratio.dropna()

#standard_deviation.head()
#sharpe_ratio.head()

m2_ratio = (sharpe_ratio*standard_deviation/TRADING_DAYS + Rf) * TRADING_DAYS

m2_ratio.plot().show()

#### Max Drawdown

A maximum drawdown (MDD) is the maximum observed loss from a peak to a trough of a portfolio, before a new peak is attained. Maximum drawdown is an indicator of downside risk over a specified time period.


$ MDD= \frac{Trough\;Value−Peak\;Value}{Peak\;Value}$



In [11]:
def max_drawdown(returns):
    cumulative_return = (returns + 1).cumprod()
    peak = cumulative_return.expanding(min_periods=1).max()
    drawdown = (cumulative_return / peak) - 1  # Change cumulative_returns to cumulative_return
    return drawdown.min()

returns = df['Close'].pct_change()
max_drawdowns = max_drawdown(returns)
print('Max drawdown is: '+ str(max_drawdowns * 100))



Max drawdown is: -10.290747938968625
